<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" align="left" /></a><br><br/>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License</a>.

# ![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)

# Word Count Lab: Building a word count application

This lab will build on the techniques covered in the Spark tutorial to develop a simple word count application. The volume of unstructured text in existence is growing dramatically, and Spark is an excellent tool for analyzing this type of data. In this lab, we will write code that calculates the most common words in the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) retrieved from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). This could also be scaled to larger applications, such as finding the most common words in Wikipedia.

### During this lab we will cover:
* Part 1: Creating a base DataFrame and performing operations
* Part 2: Counting with Spark SQL and DataFrames
* Part 3: Finding unique words and a mean value
* Part 4: Apply word count to a file

Note that for reference, you can look up the details of the relevant methods in [Spark's Python API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.sql).

## Part 1: Creating a base DataFrame and performing operations
In this part of the lab, we will explore creating a base DataFrame with `sqlContext.createDataFrame` and using DataFrame operations to count words.

### (1a) Create a DataFrame
We'll start by generating a base DataFrame by using a Python list of tuples and the `sqlContext.createDataFrame` method. Then we'll print out the type and schema of the DataFrame. The Python API has several examples for using the `createDataFrame` [method](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext.createDataFrame).

In [13]:
# works only for yarn mode
wordsDF = sqlContext.createDataFrame([('cat',), ('elephant',), ('rat',), ('rat',), ('cat', )], ['word'])
wordsDF.show()
print(type(wordsDF))
wordsDF.printSchema()

+--------+
|    word|
+--------+
|     cat|
|elephant|
|     rat|
|     rat|
|     cat|
+--------+

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- word: string (nullable = true)



### (1b) Using DataFrame functions to add an 's'
Let's create a new DataFrame from wordsDF by performing an operation that adds an 's' to each word. To do this, we'll call the [`select` DataFrame function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.select) and pass in a column that has the recipe for adding an 's' to our existing column. To generate this `Column` object you should use the [`concat` function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.concat) found in the [`pyspark.sql.functions` module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.concat). Note that concat takes in two or more string columns and returns a single string column. In order to pass in a constant or literal value like 's', you'll need to wrap that value with the [`lit` column function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.lit).

> Note: Make sure that the resulting DataFrame has one column which is named 'word'.

In [2]:
from pyspark.sql.functions import lit, concat
pluralDF = wordsDF.select(concat(wordsDF.word, lit('s')).alias('word'))
pluralDF.show()

+---------+
|     word|
+---------+
|     cats|
|elephants|
|     rats|
|     rats|
|     cats|
+---------+



### (1c) Length of each word
Now use the SQL `length` function to find the number of characters in each word. The [`length` function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.length) is found in the `pyspark.sql.functions` module.

In [3]:
from pyspark.sql.functions import length
pluralLengthsDF = pluralDF.select(length(pluralDF.word))
pluralLengthsDF.show()

+------------+
|length(word)|
+------------+
|           4|
|           9|
|           4|
|           4|
|           4|
+------------+



## Part 2: Counting with Spark SQL and DataFrames
Now, let's count the number of times a particular word appears in the 'word' column. There are multiple ways to perform the counting, but some are much less efficient than others.

A naive approach would be to call `collect` on all of the elements and count them in the driver program. While this approach could work for small datasets, we want an approach that will work for any size dataset including terabyte- or petabyte-sized datasets. In addition, performing all of the work in the driver program is slower than performing it in parallel in the workers. For these reasons, we will use data parallel operations.

### (2a) Using `groupby` and `count`
Using DataFrames, we can preform aggregations by grouping the data using the [`groupBy` function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) on the DataFrame. Using `groupBy` returns a [`GroupedData` object](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData) and we can use the functions available for `GroupedData` to aggregate the groups. For example, we can call `avg` or `count` on a `GroupedData` object to obtain the average of the values in the groups or the number of occurrences in the groups, respectively.

To find the counts of words, group by the words and then use the [`count` function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.count) to find the number of times that words occur.

In [4]:
wordCountsDF = (wordsDF
                .groupBy(wordsDF.word)
                .count()
)
wordCountsDF.show()

+--------+-----+
|    word|count|
+--------+-----+
|     rat|    2|
|     cat|    2|
|elephant|    1|
+--------+-----+



## Part 3: Finding unique words and a mean value


### (3a) Unique words
Calculate the number of unique words in `wordsDF`. You can use other DataFrames that you have already created to make this easier.

In [5]:
uniqueWordsCount = wordCountsDF.count()
print uniqueWordsCount

3


### (3b) Means of groups using DataFrames
Find the mean number of occurrences of words in `wordCountsDF`.

You should use the [`mean` GroupedData method](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.mean) to accomplish this. Note that when you use `groupBy` you don't need to pass in any columns. A call without columns just prepares the DataFrame so that aggregation functions like `mean` can be applied.

In [6]:
# Average number of occurrence
averageCount = wordCountsDF.groupBy().mean('count').head()[0]
#wordCountsDF.groupBy().select(mean().alias("count"))
print averageCount

1.66666666667


## Part 4: Apply word count to a file 
In this section we will finish developing our word count application. We'll have to build the wordCount function, deal with real world problems like capitalization and punctuation, load in our data source, and compute the word count on the new data.

### (4a) The `wordCount` function
First, define a function for word counting. You should reuse the techniques that have been covered in earlier parts of this lab. This function should take in a DataFrame that is a list of words like `wordsDF` and return a DataFrame that has all of the words and their associated counts.

In [7]:
# The wordCount function to compute number of each words in a DataFrame
def wordCount(wordListDF):
    assert(str(type(wordListDF)) == "<class 'pyspark.sql.dataframe.DataFrame'>")
    assert('word' in wordListDF.columns)
    return wordListDF.groupBy('word').count()
wordCount(wordsDF).show()

+--------+-----+
|    word|count|
+--------+-----+
|     rat|    2|
|     cat|    2|
|elephant|    1|
+--------+-----+



### (4b) Capitalization and punctuation
Real world files are more complicated than the data we have been using in this lab. Some of the issues we have to address are:
* Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
* All punctuation should be removed.
* Any leading or trailing spaces on a line should be removed.

Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces. Use the Python [regexp_replace](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.regexp_replace) module to remove any text that is not a letter, number, or space. If you are unfamiliar with regular expressions, you may want to review this [tutorial](https://developers.google.com/edu/python/regular-expressions) from Google. Also, [this website](https://regex101.com/#python) is a great resource for debugging your regular expression.

You should also use the `trim` and `lower` functions found in [pyspark.sql.functions](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions).

> Note that you shouldn't use any RDD operations or need to create custom user defined functions (udfs) to accomplish this task

In [8]:
from pyspark.sql.functions import regexp_replace, trim, col, lower

# The removePunctuation function
def removePunctuation(column):
    return trim(regexp_replace(lower(column), '[^0-9a-z\s]', '')).alias('sentence')

# A testing DataFrame for removePunctuation
sentenceDF = sqlContext.createDataFrame([('Hi, you!',),
                                         (' No under_score',),
                                         (' *      Remove punctuation and spaces   *   ',),
                                        ],['sentence'])
sentenceDF.show(truncate=False)
(sentenceDF
 .select(removePunctuation(col('sentence')))
 .show(truncate=False)
)

+--------------------------------------------+
|sentence                                    |
+--------------------------------------------+
|Hi, you!                                    |
| No under_score                             |
| *      Remove punctuation and spaces   *   |
+--------------------------------------------+

+-----------------------------+
|sentence                     |
+-----------------------------+
|hi you                       |
|no underscore                |
|remove punctuation and spaces|
+-----------------------------+



### (4c) Load a text file
For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into a DataFrame, we use the `sqlContext.read.text()` method. We also apply the recently defined `removePunctuation()` function using a `select()` transformation to strip out the punctuation and change all text to lower case. Since the file is large we use `show(15)`, so that we only print 15 lines.

Use the following code to copy local files to Hadoop.

    hadoop fs -copyFromLocal /path/to/file/shakespeare.txt /target/path/shakespeare.txt

In [9]:
fileName = 'shakespeare.txt'
shakespeareDF = sqlContext.read.text(fileName).select(removePunctuation(col('value')))
shakespeareDF.show(15, truncate=False)

+-------------------------------------------------+
|sentence                                         |
+-------------------------------------------------+
|1609                                             |
|                                                 |
|the sonnets                                      |
|                                                 |
|by william shakespeare                           |
|                                                 |
|                                                 |
|                                                 |
|1                                                |
|from fairest creatures we desire increase        |
|that thereby beautys rose might never die        |
|but as the riper should by time decease          |
|his tender heir might bear his memory            |
|but thou contracted to thine own bright eyes     |
|feedst thy lights flame with selfsubstantial fuel|
+-------------------------------------------------+
only showing

### (4d) Words from lines
Before we can use the `wordcount()` function, we have to address two issues with the format of the DataFrame:
* The first issue is that that we need to split each line by its spaces.
* The second issue is we need to filter out empty lines or words.

Apply a transformation that will split each 'sentence' in the DataFrame by its spaces, and then transform from a DataFrame that contains lists of words into a DataFrame with each word in its own row. To accomplish these two tasks you can use the `split` and `explode` functions found in [pyspark.sql.functions](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions).

Once you have a DataFrame with one word per row you can apply the [DataFrame operation](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.where) `where` to remove the rows that contain ''.

> Note that `shakeWordsDF` should be a DataFrame with one column named `word`.

In [10]:
from pyspark.sql.functions import split, explode
shakeWordsDF = (shakespeareDF
                .select(explode(split(shakespeareDF.sentence, ' ')).alias('word'))
                .where(col('word') != ''))
shakeWordsDF.show()
shakeWordsDFCount = shakeWordsDF.count()
print(shakeWordsDFCount)

+-----------+
|       word|
+-----------+
|       1609|
|        the|
|    sonnets|
|         by|
|    william|
|shakespeare|
|          1|
|       from|
|    fairest|
|  creatures|
|         we|
|     desire|
|   increase|
|       that|
|    thereby|
|    beautys|
|       rose|
|      might|
|      never|
|        die|
+-----------+
only showing top 20 rows

882996


### (4e) Count the words
We now have a DataFrame that is only words. Next, let's apply the `wordCount()` function to produce a list of word counts. We can view the first 20 words by using the `show()` action; however, we'd like to see the words in descending order of count, so we'll need to apply the [`orderBy` DataFrame method](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy) to first sort the DataFrame that is returned from `wordCount()`.

You'll notice that many of the words are common English words. These are called stopwords. In a later lab, we will see how to eliminate them from the results.

In [11]:
from pyspark.sql.functions import desc
topWordsAndCountsDF = wordCount(shakeWordsDF)
# Display the most commonly used words
topWordsAndCountsDF.orderBy(col('count').desc()).show(truncate=False)

+----+-----+
|word|count|
+----+-----+
|the |27361|
|and |26028|
|i   |20681|
|to  |19150|
|of  |17463|
|a   |14593|
|you |13615|
|my  |12481|
|in  |10956|
|that|10890|
|is  |9134 |
|not |8497 |
|with|7771 |
|me  |7769 |
|it  |7678 |
|for |7558 |
|be  |6857 |
|his |6857 |
|your|6655 |
|this|6602 |
+----+-----+
only showing top 20 rows



In [12]:
topWordsAndCountsDF.show()

+----------+-----+
|      word|count|
+----------+-----+
|       art|  915|
|      some| 1337|
|     those|  545|
|     still|  552|
|  painters|    1|
|      hope|  354|
|    travel|   33|
|     cures|    8|
|    ransom|   53|
|     spoil|   25|
|   tresses|    3|
|       few|   64|
| forgetful|    5|
|    harder|   11|
|  tripping|    6|
| soundness|    1|
|    waters|   27|
|occidental|    1|
|    marrow|    4|
|    distil|    4|
+----------+-----+
only showing top 20 rows

